<a href="https://colab.research.google.com/github/felipeddragon/dashboard-vendas/blob/main/notebooks/Dashboard_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================
# 1. IMPORTAR BIBLIOTECAS
# ============================================================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, Dropdown
from google.colab import files
from IPython.display import display, HTML
from google.colab import output
output.enable_custom_widget_manager()

sns.set(style="whitegrid")

In [2]:
# ============================================================
# 2. IMPORTAR ARQUIVO
# ============================================================
uploaded = files.upload()

Saving Zara_Sales_Analysis.csv to Zara_Sales_Analysis.csv


In [3]:
# ============================================================
# 3. CARREGAR DATASET
# ============================================================
df = pd.read_csv("Zara_Sales_Analysis.csv", sep=";")
df.columns = df.columns.str.strip()

df["price"] = pd.to_numeric(df["price"], errors="coerce")
df["Sales Volume"] = pd.to_numeric(df["Sales Volume"], errors="coerce")
df.dropna(subset=["price", "Sales Volume"], inplace=True)

df["Revenue"] = df["price"] * df["Sales Volume"]

print("Dataset carregado e processado!")

Dataset carregado e processado!


In [4]:
# ============================================================
# 4. EXIBIR KPIs PRINCIPAIS
# ============================================================
def mostrar_kpis(df):
    receita = df["Revenue"].sum()
    vendas = df["Sales Volume"].sum()
    produtos = df["name"].nunique()

    display(pd.DataFrame({
        "Indicador": ["Receita Total", "Unidades Vendidas", "Produtos Únicos"],
        "Valor": [f"R$ {receita:,.2f}", f"{vendas:,}", produtos]
    }))

mostrar_kpis(df)

,Indicador,Valor
0,Receita Total,"R$ 38,988,476.48"
1,Unidades Vendidas,"459,573"
2,Produtos Únicos,194


In [5]:
# ============================================================
# 5. FUNÇÃO PARA GERAR GRÁFICOS DINÂMICOS
# ============================================================
def gerar_graficos(metrica):

    plt.figure(figsize=(11, 5))

    # --------------------------------------------------------
    # 1. Receita total por produto
    # --------------------------------------------------------
    if metrica == "Top 10 - Receita":
        top = df.groupby("name")["Revenue"].sum().nlargest(10)
        sns.barplot(x=top.values, y=top.index, palette="viridis")
        plt.title("Top 10 Produtos por Receita")

    # --------------------------------------------------------
    # 2. Produtos mais vendidos
    # --------------------------------------------------------
    elif metrica == "Top 10 - Unidades Vendidas":
        top = df.groupby("name")["Sales Volume"].sum().nlargest(10)
        sns.barplot(x=top.values, y=top.index, palette="magma")
        plt.title("Top 10 Produtos Mais Vendidos")

    # --------------------------------------------------------
    # 3. Distribuição de Preços
    # --------------------------------------------------------
    elif metrica == "Distribuição de Preços":
        sns.histplot(df["price"], kde=True)
        plt.title("Distribuição dos Preços dos Produtos")
        plt.xlabel("Preço (R$)")

    # --------------------------------------------------------
    # 4. Dispersão preço x vendas
    # --------------------------------------------------------
    elif metrica == "Correlação: Preço x Vendas":
        sns.scatterplot(x=df["price"], y=df["Sales Volume"])
        plt.title("Correlação entre Preço e Unidades Vendidas")
        plt.xlabel("Preço (R$)")
        plt.ylabel("Vendas")

    # --------------------------------------------------------
    # 5. Receita acumulada
    # --------------------------------------------------------
    elif metrica == "Receita Acumulada (Crescente)":
        acumulado = df.sort_values("Revenue", ascending=False)["Revenue"].cumsum()
        plt.plot(acumulado.values)
        plt.title("Receita Acumulada (Crescente por Produto)")
        plt.ylabel("Receita acumulada")
        plt.xlabel("Produtos (ordenados)")

    plt.tight_layout()
    plt.show()

In [6]:
# ============================================================
# 6. MENU INTERATIVO AVANÇADO
# ============================================================
menu = Dropdown(
    options=[
        "Top 10 - Receita",
        "Top 10 - Unidades Vendidas",
        "Distribuição de Preços",
        "Correlação: Preço x Vendas",
        "Receita Acumulada (Crescente)"
    ],
    value="Top 10 - Receita",
    description="Visualização:"
)

interact(gerar_graficos, metrica=menu)


interactive(children=(Dropdown(description='Visualização:', options=('Top 10 - Receita', 'Top 10 - Unidades Ve…

<function __main__.gerar_graficos(metrica)>